# SQLAlchemy Primer

Goals:

- creating SQL tables;

- populating tables with data;

- simple select data queries;

- updating data and even tables;

- doing all the above in OOP style - moving away from "SQL texts"

## Table of Contents

- [So be a table](#so-be-a-table)

- [Table normalisation](#table-normalisation)

- [References](#references)

In [ ]:
import sqlalchemy as sqla
from sqlalchemy.orm import DeclarativeBase
import sqlalchemy.ext.asyncio as async_sqla


print(f"SQLAlchemy version: {sqla.__version__}")

Previously touched:

- asyncio code takes precedence throughout the tutorial;

- SQLAlchemy >= 2;

- asynchronous DB API's: [aiosqlite](https://aiosqlite.omnilib.dev/en/stable/) for now, but other connectors, like [asyncpg](https://magicstack.github.io/asyncpg/current/), will be covered later in advanced topics.

In [ ]:
url: str = "sqlite+aiosqlite:///:memory:"
async_engine = async_sqla.create_async_engine(url=url, echo=True)

metadata = sqla.MetaData()  # will be explained later

print(f"Engine: {async_engine}")
print(f"MetaData: {metadata}")

### So be a table

**Here and below** mostly everything will be put simply, please do not forget about this disclaimer and don't hesitate to do your own researches.

SQL is about relational theory in practice. A relation is mapped to a table in the SQL world (but some authors say these entities are not the same). You may read more about ["The SQL Standard – ISO/IEC 9075:2023 (ANSI X3.135)"](https://blog.ansi.org/sql-standard-iso-iec-9075-2023-ansi-x3-135/).

So, in SQLAlchemy there is a `class sqlalchemy.schema.Table` for mapping an object/instance of this class to a table in a relational database management system (RDBMS). It needs a table name and metadata. Generally, metadata are data that describe data like the format, structure and so on. In SQLAlchemy, the term “metadata” typically refers to the [MetaData](https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.MetaData) construct, which is a collection of information about the tables, columns, constraints, and other [DDL](https://docs.sqlalchemy.org/en/20/glossary.html#term-DDL) (Data Definition Language) objects that may exist in a particular database ([link](https://docs.sqlalchemy.org/en/20/glossary.html#term-database-metadata)).

Time for the action - following the SQLAlchemy [tutorial page](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html) with tables and metadata. First, direct table instance construction -> creating a Table instance != creating the corresponding SQL table!

In [ ]:
# Attention! The cell is meant to be run once

# table direct construction
users_table = sqla.Table(
    "users",
    metadata,
    sqla.Column("ID", sqla.Integer, primary_key=True, autoincrement=True),
    sqla.Column("full name", sqla.String(length=100), nullable=False),
)
users_table

In [ ]:
# However, in case you need to rerun the previous cell,
# you can uncomment the following line:
# metadata.remove(users_table)

print(f"Users table: {users_table}")
print(f"Users keys: {users_table.c.keys()}")
print(f"Users primary key: {users_table.primary_key}")

print(f"\nMetaData.tables: {metadata.tables}")

Let's insert some data to the "Users" table, but before the leap, a simple `SELECT * FROM Users` statement as a [smoke test](https://en.wikipedia.org/wiki/Smoke_testing_(software)).

In [ ]:
async with async_engine.connect() as async_conn:
    await async_conn.execute(users_table.select())

Whoa, the table "Users" does not exist, how so if we can deal with users_table object? The explanation is simple: `users_table` object has no the corresponding relation in the database (supposing in-memory [tabula rasa](https://en.wikipedia.org/wiki/Tabula_rasa) mode).

An important step - to establish the mapping and synchronise relations with objects and metadata will help us out. Trying again:

In [ ]:
async with async_engine.begin() as transaction:
    await transaction.run_sync(metadata.create_all)

Notes on the above code:

- The `metadata.create_all(engine)` execution is for a synchronous code. Within an asynchronous code we need to call `create_all` within the `run_sync` method which allows running synchronous callables.

- the `begin()` method starts a transaction which is a unit of work performed on a database. If a transaction fails, all work is rolled back to the initial state. Feel free to read more about [SQL transactions](https://www.tutorialspoint.com/sql/sql-transactions.htm).

The SQL expression of interest:

```sql
CREATE TABLE users (
	"ID" INTEGER NOT NULL, 
	"full name" VARCHAR(100) NOT NULL, 
	PRIMARY KEY ("ID")
)
```

The previous statement can be rewritten into something like that (but it is SQLite syntax, meaning the less general solution):

```sql
CREATE TABLE Users (
	"ID" INTEGER PRIMARY KEY,
	"full name" VARCHAR(100) NOT NULL
);
```

Memos:

- **SQL is case-insensitive**;

- AUTOINCREMENT is not needed in SQLite ([proof](https://www.sqlite.org/autoinc.html))

Seems promisible, but looking does not mean being, only a retest will reveal the truth.

In [ ]:
async with async_engine.connect() as conn:
    stmt = users_table.select()
    result = await conn.execute(stmt)
    print(f"SELECT ALL FROM Users; -> {result.fetchall()}")

Worked like a charm.

Note that SQLAlchemy composed the expression to select all attributes/columns from "Users" table in this explicit way:

```sql
SELECT users."ID", users."full name"
FROM users
```

Very soon enough other SQL expressions to reach the same goal will be demonstrated in the code.

Now we are able to insert data.

In [ ]:
async with async_engine.begin() as ta:
    stmt = users_table.insert()
    # can you guess why passing a dictionary (args), not kwargs?
    await ta.execute(
        stmt.values(
            [
                {"full name": "John Doe"},
                {"full name": "Jane Lane"}
            ]
        )
    )
    # rememeber that id attribute gets autoincrement'ed

The previous statement can be written (again, SQLite syntax) as:

```sql
INSERT INTO Users ('full name') VALUES ('John Doe'), ('Jane Lane');
```

More about "SQL\[Lite\] INSERT" statement can be found [here](https://www.sqlitetutorial.net/sqlite-insert/). Since SQLite v3.7.11 bulk inserts (multiple values within a single "INSERT" statement) are doable.

![SQLite3 Bulk Insert](./sqlite3_bulk_insert.png "SQLite3 Bulk Insert picture")

Again, SQL is case-insensitive. Capitalising SQL keywords is a convention to make them stand out. If you don't like pressing "Shift" button, consider using the "Capslock".

In [ ]:
async with async_engine.connect() as async_conn:
    result = await async_conn.execute(users_table.select())
    print(f"SELECT * FROM Users; -> {result.fetchall()}")

Another form of selecting all the attributes/columns from a table (here "Users") is `SELECT * FROM Users;` statement.

In [ ]:
async with async_engine.connect() as async_conn:
    stmt: str = "SELECT * FROM Users;"
    result = await async_conn.execute(sqla.text(stmt))
    print(f"`{stmt}` -> {result.fetchall()}")

Thanks to the [SQLAlchemy asyncio docs](https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html) for being rich enough especially with examples.

So, basic operations are left behind, but there are some imperfections:

- the "ID" attribute is in upper case, how about having all names in lower case;

- "full name" attribute is a composite one and in database design atomic (indivisible) attributes are preferable, so this field is to be split into two columns like "first_name" and "last_name";

- certainly data must not be lost and the migration should be done gracefully.

### Table normalisation

So, we have three issues to wipe out. To introduce a declarative form of defining table MetaData, we shall to:

1. declare another class with an appropriate structure;

2. create a temporary database with the same structure, but another name like "tmp" or something;

3. move data from the already populated "Users" table into the "tmp" one and then from the temporary database

It is much likely a costly and silly solution to be avoided, but it is still practice. If there is a solution and there are no better options, use what you have unless you find a better solution.

In [ ]:
from sqlalchemy.orm import (
    Mapped,
    mapped_column,
)

Now, the tricky part is that "Users" table already exists in the database.
That is why the table will initially have "tmp" name.

In [ ]:
new_metadata = sqla.MetaData()

# InvalidRequestError:
# Cannot use 'DeclarativeBase' directly as a declarative base class.
# Create a Base by creating a subclass of it.
class Base(DeclarativeBase):
    """SQLAlchemy Declarative Base class."""


class Tmp(Base):
    __tablename__ = "tmp"
    metadata = new_metadata  # usually SQLAlchemy assigns it for you

    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    first_name: Mapped[str] = mapped_column(sqla.String(30), nullable=False)
    last_name: Mapped[str] = mapped_column(sqla.String(30), nullable=False)


tmp_table = Tmp()
tmp_table  # ceci n'est pas une table

Extraction from [SQLAlchemy docs: declarative tables](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html)

> Above, when Declarative processes each class attribute, each [mapped_column()](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.mapped_column) will derive additional arguments from the corresponding [Mapped](https://docs.sqlalchemy.org/en/20/orm/internals.html#sqlalchemy.orm.Mapped) type annotation on the left side, if present. Additionally, Declarative will generate an empty mapped_column() directive implicitly, whenever a Mapped type annotation is encountered that does not have a value assigned to the attribute (this form is inspired by the similar style used in Python dataclasses); this mapped_column() construct proceeds to derive its configuration from the Mapped annotation present.

In [ ]:
async with async_engine.begin() as ta:
    await ta.run_sync(new_metadata.create_all)

Cool, it worked, now the action can be done in one loop over two tables:

1. select all data from the old table;
2. row processing
3. insert the processed data into the new table

Then the old table can be easilly dropped away, and the new one is renamed into "Users".

In [ ]:
# README ATTENTIVELY

async with async_engine.connect() as async_conn:
    rows = await async_conn.execute(users_table.select())
    new_rows: list[tuple[int, str]] = []
    for index, (id_, full_name) in enumerate(rows, 1):  # id is reserved in Python
        print(f"Row_{index}: id={id_} -> full_name={full_name}")
        new_row: tuple[int, str] = (id_, full_name.strip().split(' '))
        print(f"NewRow_{index}: {new_row}")
        new_rows.append(new_row)
    # maybe there is a better way
    session = async_sqla.async_sessionmaker(async_engine, expire_on_commit=True)
    async with session() as async_session:
        async_session.add_all(
            [
                Tmp(
                    id=id_,
                    first_name=fname,
                    last_name=lname,
                )
                for id_, (fname, lname) in new_rows
            ]
        )
        await async_session.commit()

The "tmp" table has been populated with processed data from the "users" table.

The Red Letter moment...

In [ ]:
async_session = async_sqla.async_sessionmaker(async_engine)
async with async_session() as session:
    result = await session.execute(sqla.select(Tmp))
    scalars = result.scalars()
    print(f"'tmp' table scalars: {scalars}")
    for row in scalars:
        entry = (row.id, row.first_name, row.last_name)
        print(f"tmp row: {entry}")

Yes, eventually, it was tiresome and finally over...ah, renaming the "tmp" table, yep, holding the horses.

First, removing the "Users" table only.

In [ ]:
async with async_engine.begin() as ta:
    await ta.run_sync(
        users_table.drop,
        checkfirst=True
    )

Second, checking that the table "Users" is dropped (the error is coming).

In [ ]:
async with async_engine.connect() as conn:
    stmt = "SELECT * FROM Users;"
    await conn.execute(sqla.text(stmt))

But the "tmp" table exists.

In [ ]:
async with async_engine.connect() as async_conn:
    result = await async_conn.execute(sqla.select(Tmp))
    print(f"SELECT * FROM tmp; -> {result.fetchall()}")

Finally, we can easily change or ALTER the name of "tmp" table into "users".

In [ ]:
async with async_engine.begin() as ta:
    stmt = "ALTER TABLE tmp RENAME TO users;"
    await ta.execute(sqla.text(stmt))

And if selecting all columns (and rows) from the "users" table, we shall see the data previously inhabitating the "tmp" table, which are the same tables, so here two points are checked:

1. "tmp" is really renamed and accessed by the new "users" name

2. no data missing

In [ ]:
async_session = async_sqla.async_sessionmaker(async_engine)
async with async_session() as session:
    stmt = "select * from users;"
    result = await session.execute(sqla.text(stmt))
    for row in result.fetchall():
        print(row)

Everything went fine...or did it? There is not a small hindrance: metadata instances got outdated. The data they hold are obsolete.

In [ ]:
print(f"NEW METADATA: {new_metadata.tables}\n")
print(f"OLD METADATA: {metadata.tables}\n")

We cannot address `Tmp` class anymore and this is normal since it has `__tablename__ = "tmp"` attribute.

In [ ]:
async_session = async_sqla.async_sessionmaker(async_engine)
async with async_session() as session:
    result = await session.execute(sqla.select(Tmp))
    for row in result.scalars():
        print(row)

But you can't just reassign it with a new name because metadata are obsolete - mapping is corrupted.

In [ ]:
Tmp.__tablename__ = "users"

async_session = async_sqla.async_sessionmaker(async_engine)
async with async_session() as session:
    result = await session.execute(sqla.select(Tmp))
    for row in result.scalars():
        print(row)

If taking the Stack Overflow question ["How to rename an existing table"](https://stackoverflow.com/questions/49550784/how-to-rename-an-existing-table), we had better to migrate data via tools like:

- [alembic](https://alembic.sqlalchemy.org/en/latest/)

- [sqlalchemy-migrate](https://sqlalchemy-migrate.readthedocs.io/en/latest/)

Migrations are something to be palpated later, but inevitably because it is a must-have when dealing with databases and ORM stuff especially.

Let's try another feature which is called **reflection**. Reflection allows to get SQLAlchemised objects mapped from the structure/schema of a database. Instead of defining classes manually, SQLAlchemy does the kitchen stuff for us and we can get the instances of corresponding entities in the database where not only tables live.

Illustrating the reflection with examples.

In [ ]:
meta = sqla.MetaData()  # Not that META, certainly
meta.reflect(bind=async_engine)
users_table = meta.tables["users"]
users_table

Ok, the hint is taken, the solution above is about synchronous way to reflect database structure, so trying the async-aware approach.

In [ ]:
def reflect_table(conn_, name: str) -> sqla.Table:
    """Reflects the table `name` from the DB via connection `conn_`."""

    return sqla.Table(
        name,
        sqla.MetaData(),
        autoload_with=conn_
    )


async with async_engine.connect() as conn:
    users_table: sqla.Table = await conn.run_sync(
        reflect_table, "users"
    )
    print(f"Table name -> {users_table.name}")
    print(f"Table columns -> {users_table.c.items()}")
    print(f"Selecting data from {users_table} table.")

And now we can play with "users_table" variable which references to the Table object reflected from "Users" table of the database which in its turn represents the structure of this table (see "Table columns" printed message from the code above)...and it's awesome.

In [ ]:
async with async_engine.connect() as async_conn:
    result = await async_conn.execute(users_table.select())
    print(f"SELECT * FROM Users; -> {result.fetchall()}")

Yep, it worked, a little much, so that is all for this note.

Totals:

- basic operations for asyncio-oriented code: raw SQL statements and via SQLAlchemy utilities (just enough for the start);

- coherence between DB entities and their counterparts in the code is vital: `ALTER TABLE` case showed a pit to fall easily into;

- reflection is cool, but it does not cancel the necessity to keep your code clean: it's better to see the structure through the classes as they are, not tracing the actual state of matters throughout the code.

### References

- [SQLAlchemy asyncio tutorial](https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html)

- [SQLite FAQ](https://www.sqlite.org/faq.html)

- SQL statements:

  - [W3Schools: CREATE TABLE](https://www.w3schools.com/sql/sql_create_table.asp)

  - [W3Schools: INSERT INTO 'tablename' VALUES ...](https://www.w3schools.com/sql/sql_insert.asp)

  - [W3Schools: SELECT ... FROM 'tablename'](https://www.w3schools.com/sql/sql_select.asp)

  - ALTER TABLE or RENAME TABLE variants [(Tutorials Point)](https://www.tutorialspoint.com/sql/sql-rename-table.htm)

- Problem-addressing references:

  - [SQLAlchemy: reflection](https://docs.sqlalchemy.org/en/20/core/reflection.html)

  - [GitHub: SQLAlchemy Runtime Inspection API doesn't support AsyncEngine #6121](https://github.com/sqlalchemy/sqlalchemy/issues/6121)

  - [Stack Overflow: Get existing table using SQLAlchemy MetaData](https://stackoverflow.com/questions/44193823/get-existing-table-using-sqlalchemy-metadata)

  - [Stack Overflow: How to delete a table in SQLAlchemy?](https://stackoverflow.com/questions/35918605/how-to-delete-a-table-in-sqlalchemy)

  - [Youtube: Using SQLAlchemy Asynchronously With AsyncIO (SQLAlchemy 2.0)](https://www.youtube.com/watch?v=hkvngd_BUrY)